# Amazon Bedrock AgentCore Policy - 入門デモ

## 概要

Amazon Bedrock AgentCore Policy のハンズオンデモへようこそ！このノートブックでは、AI エージェントとツールの連携に対するポリシーベースの決定論的制御を設定およびテストする完全なワークフローをガイドします。

### AgentCore Policy とは？

Amazon Bedrock AgentCore Policy を使用すると、開発者はエージェント操作の周りに保護境界（「セーフティボックス」）を作成することで、AI エージェントとツールの連携に対するセキュリティ制御を定義および適用できます。AI エージェントは複雑な問題を解決するために動的に適応できますが、この柔軟性にはセキュリティ上の課題が伴います：

- **データ漏洩**: エージェントが意図せず機密情報を公開する可能性
- **ビジネスルール違反**: エージェントがビジネスルールを誤解したりバイパスしたりする可能性
- **権限の逸脱**: エージェントが意図された範囲外で動作する可能性

Policy は AgentCore Gateway を通じてインバウンドエージェントトラフィックをインターセプトし、ツールへのアクセスを許可する前に、定義されたポリシーに対して各リクエストを評価します。

### 主なメリット

- **宣言的セキュリティ**: コードではなく Cedar 言語でポリシーを定義
- **ランタイム適用**: ポリシーはリアルタイムで評価
- **きめ細かい制御**: 粗い制限から詳細なアクセス制御/認可まで
- **関心の分離**: セキュリティロジックはエージェントコードの外部に存在
- **エンタープライズスケール**: 自律型エージェントを本番環境で安全にデプロイ

---

## デモアーキテクチャ

```
┌─────────────┐
│   AI Agent  │
└──────┬──────┘
       │
       │ Tool Call Request
       ▼
┌─────────────────────┐
│  AgentCore Gateway  │
│  + OAuth Auth       │
└──────┬──────────────┘
       │
       │ Policy Check
       ▼
┌─────────────────────┐
│   Policy Engine     │
│   (Cedar Policies)  │
└──────┬──────────────┘
       │
       │ ALLOW / DENY
       ▼
┌─────────────────────--┐
│   Gateway Targets     │
│                       │    
└─────────────────────--┘
```

---

## 学習内容

このデモでは、保険引受業務を支援するツールを持つエージェントをセットアップします：

1. **インフラストラクチャのセットアップ**: 保険申請の作成、リスクモデルの呼び出し、保険請求の承認のための Lambda ターゲットを持つ Gateway を作成
2. **Policy Engine の作成**: Gateway 用の Policy Engine を初期化
3. **ポリシーの定義**: アクセスを制御する Cedar ポリシーを作成
4. **適用のテスト**: 実際のエージェントリクエストでポリシーが機能することを確認
5. **結果の理解**: ALLOW と DENY のシナリオを解釈

---

## 前提条件

開始する前に、以下を確認してください：

- AWS CLI が適切な認証情報で設定されていること
- Python 3.10+ と boto3 がインストールされていること
- `bedrock_agentcore_starter_toolkit` パッケージがインストールされていること
- AWS Lambda へのアクセス権限があること（ターゲット関数用）

---

## デモシナリオ: 保険引受処理

ポリシー制御を備えた**保険引受処理システム**を実装します：

- **ツール**: 
       - `ApplicationTool` - 地域と補償額を指定して保険申請を作成
       - `RiskModelTool` - ガバナンス制御付きで外部リスクスコアリングモデルを呼び出し、API 分類とデータガバナンス承認が必要
       - `ApprovalTool` - 高額または高リスクの引受決定を承認、パラメータは claim_amount と risk_level


それでは始めましょう！

---

# ステップ 0: 環境セットアップ

まず、環境を確認し、必要なライブラリをインポートします。

In [ ]:
# Install requirements
%pip install -r requirements.txt

In [ ]:
# Import required libraries
import sys
from pathlib import Path
import boto3
import json
import logging

In [ ]:
# Add the scripts directory to Python path
scripts_dir = Path.cwd() / "scripts"
if str(scripts_dir) not in sys.path:
    sys.path.insert(0, str(scripts_dir))

# Verify region
session = boto3.Session()
region = session.region_name or "us-east-1"

# Verify AWS credentials
try:
    sts = session.client("sts")
    identity = sts.get_caller_identity()
    print("✅ AWS 認証情報を確認しました")
    print(f"   アカウント: {identity['Account']}")
    print(f"   ユーザー/ロール: {identity['Arn']}")
except Exception as e:
    print(f"❌ AWS 認証情報エラー: {e}")
    print("   AWS CLI を設定してください: aws configure")

---

# ステップ 1: AgentCore Gateway 用のターゲット関数を作成

Gateway をセットアップする前に、エージェントのツールターゲットとして機能する関数が必要です。

## Lambda ターゲットとは？

Lambda ターゲットは、AI エージェントが Gateway を通じて呼び出すことができるバックエンド関数です。今回の場合、エージェントが保険引受タスクを実行するために、ApplicationTool、RiskModelTool、ApprovalTool 用の 3 つの Lambda 関数をセットアップします。

### CLI で作成（以下のセルを実行）
以下のスクリプトを実行すると、AWS アカウントに 3 つの Lambda がデプロイされます：

1. Application Tool: 簡易版申請作成（デモ用のモック）
 申請者の地域と補償額を指定して保険申請を作成
 パラメータ:
 - applicant_region: 顧客の地理的地域
 - coverage_amount: 希望する保険補償額

2. Risk Model Tool: 簡易版リスクモデルアクセス（デモ用のモック）
 リスクスコアリングモデルを呼び出し、評価を返す
 パラメータ:
 - API_classification: API 分類（public、internal、restricted）
 - data_governance_approval: データガバナンスがモデル使用を承認したかどうか

3. Approval Tool: 保険承認プロセス（デモ用のモック）
 引受決定と請求額を承認
 パラメータ:
 - claim_amount: 保険請求/補償額
 - risk_level: リスクレベル評価（low、medium、high、critical）

In [ ]:
%run scripts/lambda-target-setup/deploy_lambdas.py 

---

# ステップ 2: AgentCore Gateway のセットアップ

OAuth 認証を使用して AgentCore Gateway を作成し、Lambda 関数をターゲットとして接続します。

## 作成されるもの

1. **OAuth 認可サーバー**: 認証用の Cognito ベースの OAuth
2. **AgentCore Gateway**: MCP プロトコルをサポートするメイン Gateway
3. **Lambda ターゲット**: スキーマ定義付きで接続された Lambda 関数
4. **設定ファイル**: 後で使用するために保存されるすべての接続詳細

## Gateway 設定

Gateway は以下で設定されます：
- **プロトコル**: MCP（Model Context Protocol）
- **認証**: Cognito 経由の OAuth 2.0
- **ターゲット**: ApplicationTool、RiskModelTool、ApprovalTool の Lambda 関数 - ターゲットスキーマが Gateway に提供されます

In [ ]:
# Run the gateway setup script
print("🚀 AgentCore Gateway をセットアップ中...\n")
print("以下を実行します:")
print("  1. OAuth 認可サーバーを作成 (Cognito)")
print("  2. AgentCore Gateway を作成")
print("  3. Lambda をターゲットとしてアタッチ")
print("  4. 設定を config.json に保存")
print("\n" + "=" * 60)

# Run the gateway setup script
%run scripts/setup_gateway.py

### Gateway 設定の確認

作成された Gateway 設定を読み込んで確認しましょう：

In [ ]:
# Load gateway configuration
gateway_config_file = "config.json"

with open(gateway_config_file, "r") as f:
    gateway_config = json.load(f)

print("✅ Gateway Configuration Loaded\n")
print("=" * 60)
print(f"ゲートウェイ ID:  {gateway_config['gateway']['gateway_id']}")
print(f"ゲートウェイ ARN: {gateway_config['gateway']['gateway_arn']}")
print(f"ゲートウェイ URL: {gateway_config['gateway']['gateway_url']}")
print(f"リージョン:      {gateway_config['region']}")
print("\nOAuth Configuration:")
print(f"  Client ID:  {gateway_config['gateway']['client_info']['client_id']}")
print(f"  Token URL:  {gateway_config['gateway']['client_info']['token_endpoint']}")
print("=" * 60)

# Store for later use
GATEWAY_ARN = gateway_config["gateway"]["gateway_arn"]
GATEWAY_ID = gateway_config["gateway"]["gateway_id"]
GATEWAY_URL = gateway_config["gateway"]["gateway_url"]

### Gateway 上で作成されたツールを使用するエージェントを実行

In [ ]:
# Import the agent session
from scripts.agent_with_tools import AgentSession

# Use the agent within a context manager (this handles setup and cleanup automatically)
with AgentSession() as session:
    # The agent will list all available tools during setup

    # Now you can invoke the agent with different prompts
    response1 = session.invoke("What tools do you have access to?")

    response2 = session.invoke("Create an application for US region with $5M coverage")

    response3 = session.invoke(
        "Invoke the risk model with public API classification and data governance approval set to true"
    )

    response4 = session.invoke(
        "Approve underwriting for $75000 claim with medium risk level"
    )

# The session is automatically cleaned up when exiting the 'with' block
print("=" * 60)
print(
    f"🚀 The agent has access to the following tools configured on the Gateway: {response1}\n"
)
print(f"🚀 The agent can create applications without any limits: {response2}\n")
print(f"🚀 The agent can invoke the risk model: {response3}\n")
print(f"🚀 The agent is able to approve the insurance claims: {response4}\n")
print("=" * 60)

---

# ステップ 3: Policy Engine とポリシーの作成

Gateway 上のツールへのアクセスを制御するために、Cedar ポリシーを持つ Policy Engine を作成します。

## Policy Engine とは？

Policy Engine は、Cedar ポリシーに対してリクエストをリアルタイムで評価します。2 つのモードで動作します：
- **LOG_ONLY**: 評価するがブロックしない（テスト用）
- **ENFORCE**: 非準拠のリクエストを積極的にブロック（本番用）

Gateway が Policy Engine に関連付けられると、デフォルトのアクションは拒否になり、特定のポリシーがアクセスを許可しない限りアクセスできません。空の Policy Engine では、Gateway 上のすべてのツールへのアクセスが許可されません。

### Policy Engine の作成

まず、Cedar ポリシーを保持する Policy Engine を作成します：

In [ ]:
# Import PolicyClient from AgentCore Starter Toolkit
from bedrock_agentcore_starter_toolkit.operations.policy.client import PolicyClient

policy_client = PolicyClient(region_name=region)
policy_client.logger.setLevel(logging.INFO)

# Create a Policy Engine
print("🔧 Creating Policy Engine...")
engine = policy_client.create_or_get_policy_engine(
    name="InsurancePolicyEngine",
    description="Policy engine for insurance underwriting governance",
)
print(f"✓ Policy Engine: {engine['policyEngineId']}\n")

In [ ]:
# Save Policy Engine in the configuration file
with open("config.json", "r") as f:
    config = json.load(f)

# Add policy engine information (without removing existing data)
config["policy_engine_id"] = engine["policyEngineId"]
config["policy_engine_arn"] = engine["policyEngineArn"]

# Write back the updated config
with open("config.json", "w") as f:
    json.dump(config, f, indent=2)

print("✅ Policy engine information added to config.json")

In [ ]:
# Attach Policy Engine to the Gateway
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient

gateway_client = GatewayClient(region_name=region)
gateway_client.logger.setLevel(logging.INFO)

gateway_client.update_gateway_policy_engine(
    gateway_identifier=config["gateway"]["gateway_id"],
    policy_engine_arn=engine["policyEngineArn"],
    mode="ENFORCE",
)
print("✓ Policy Engine attached to Gateway\n")

### Policy Engine が Gateway に接続されたので、デフォルトでツールはブロックされます。list tools API 呼び出し時も、エージェントが Gateway を通じてアクセスしようとする時も同様です

In [ ]:
# Import the agent session
from scripts.agent_with_tools import list_available_tools, fetch_access_token

client_info = config["gateway"]["client_info"]

CLIENT_ID = client_info["client_id"]
CLIENT_SECRET = client_info["client_secret"]
TOKEN_URL = client_info["token_endpoint"]
access_token = fetch_access_token(CLIENT_ID, CLIENT_SECRET, TOKEN_URL)

print("=" * 60)
print("🚀 The agent has access to the following tools configured on the Gateway: \n")
print(list_available_tools(config["gateway"]["gateway_url"], access_token))

### Cedar ポリシーの作成

補償額が 100 万ドル未満の場合に保険申請の作成を許可する Cedar ポリシーを作成します：

## ポリシールール

```cedar
permit(
  principal,
  action == AgentCore::Action::"ApplicationToolTarget___create_application",
  resource == AgentCore::Gateway::"<gateway-arn>"
) when {
  context.input.coverage_amount <= 1000000
};
```

これが意味すること：
- 100 万ドル以下の保険申請を作成: **許可**
- 100 万ドルを超える保険申請を作成: **拒否**

In [ ]:
# Create Cedar policy
print("\n📝 Creating Cedar Policy...")
print(f"   Policy Engine ID: {engine['policyEngineArn']}")
GATEWAY_ARN = config["gateway"]["gateway_arn"]

# Define the Cedar policy statement
cedar_statement = (
    f"permit(principal, "
    f'action == AgentCore::Action::"ApplicationToolTarget___create_application", '
    f'resource == AgentCore::Gateway::"{GATEWAY_ARN}") '
    f"when {{ context.input.coverage_amount <= 1000000 }};"
)

policy = policy_client.create_or_get_policy(
    policy_engine_id=engine["policyEngineId"],
    name="create_application_policy",
    description="Allow application creation under $1M",
    definition={"cedar": {"statement": cedar_statement}},
)
print(f"✓ Policy: {policy['policyId']}\n")

# Save to config
config["policy_id"] = policy["policyId"]
with open("config.json", "w") as f:
    json.dump(config, f, indent=2)

---

# ステップ 4: AI エージェントでポリシー適用をテスト

ここからがエキサイティングな部分です - 実際の AI エージェントでポリシーをテストしましょう！
create_application ツールに接続したポリシーにより、このツールは Gateway でリストされ、エージェントから呼び出せるようになっていることにも注目してください。

## テストシナリオ

2 つのシナリオをテストします：

### テスト 1: 許可シナリオ
- **リクエスト**: 補償額 $750,000 の申請を作成
- **期待**: ポリシーが許可、Lambda が実行、申請が作成される
- **理由**: $750K <= $100 万（ポリシー制限内）

In [ ]:
# Use the agent within a context manager (this handles setup and cleanup automatically)
with AgentSession() as session:
    # The agent will list all available tools during setup

    # Now you can invoke the agent with different prompts
    response1 = session.invoke("What tools do you have access to?")

    response2 = session.invoke(
        "Create an application for US region with $750,000 coverage"
    )

### テスト 2: 拒否シナリオ
- **リクエスト**: 補償額 $150 万の申請を作成
- **期待**: ポリシーがブロック、Lambda は実行されない
- **理由**: $150 万 > $100 万（ポリシー制限を超過）

In [ ]:
with AgentSession() as session:
    # The agent will list all available tools during setup

    response2 = session.invoke(
        "Create an application for US region with $1.5M coverage"
    )

## クリーンアップ

<div style="background-color: #d1ecf1; border-left: 4px solid #0c5460; padding: 10px; margin: 10px 0; color: #000;">
    <strong style="color: #000;">注意:</strong> 02-Natural-Language-Policy-Authoring/NL-Authoring-Policy.ipynb は、このデモでセットアップした Gateway を再利用します。2 番目のラボで NL2Cedar 機能をテストした後に、クリーンアップステップを実行することをお勧めします。
</div>


Policy Engine とポリシーのリソースをクリーンアップするには、以下の順序で行います：
1. update_gateway CLI を使用して空の Policy Engine を渡すことで、Gateway 上の Policy Engine の関連付けを削除

2. Policy Engine 内のすべてのポリシーを削除

3. Policy Engine を削除

In [ ]:
from bedrock_agentcore_starter_toolkit.operations.gateway.client import GatewayClient
from bedrock_agentcore_starter_toolkit.operations.policy.client import PolicyClient

with open("config.json", "r") as f:
    config = json.load(f)

# Clean up Policy Engine first
print("🧹 Cleaning up Policy Engine...")
policy_client = PolicyClient(region_name=config["region"])
policy_client.cleanup_policy_engine(config["policy_engine_id"])
print("✓ Policy Engine cleaned up\n")

# Then clean up Gateway
print("🧹 Cleaning up Gateway...")
gateway_client = GatewayClient(region_name=config["region"])
gateway_client.cleanup_gateway(
    config["gateway"]["gateway_id"], config["gateway"]["client_info"]
)
print("✅ Cleanup complete!")